# 1. Sort by names A - -Z

In [2]:
import pandas as pd
from openpyxl import load_workbook

file_path = "Attendance Check Example.xlsx"
df = pd.read_excel(file_path, header=1, engine='openpyxl')

df_sorted = df.sort_values(by='Name')

wb = load_workbook(file_path)
ws = wb.active

# Clear original data from row 3, column 2 onward
for row in range(3, ws.max_row + 1):
    for col in range(2, ws.max_column + 1):
        ws.cell(row=row, column=col).value = None

# Write sorted data from row 3, column 2 onward
for index, row in enumerate(df_sorted.itertuples(index=False), start=3):
    for col_num, value in enumerate(row[1:], start=2):
        ws.cell(row=index, column=col_num, value=value)

wb.save(file_path)

print(f"Data written from row 3, keeping column 1 unchanged, saved to: {file_path}")


Data written from row 3, keeping column 1 unchanged, saved to: Attendance Check Example.xlsx


# 2. Verify count of absences and names before writing to the file


In [9]:
import pandas as pd

# Define the list of names from OCR extraction
names_list = [
    "Amy", "Alice", "Bob", "Charlie", 
    "Hank", "Ivy", "Jack", "Kate", "Leo", "Mia", "Nina", "Oliver", 
    "Paul", "Quinn", "Rachel", "Sam", "Tina", "Uma", "Victor", 
    "Wendy", "Xander", "Ya", "Zoe", "Aaron", "Bella", "Cody", 
    "Diana", "Ethan", "Fiona", "George", "Hazel", "Isaac", 
    "Jasmine", "Kevin", "Luna", "Mason", "Rudy", "Sara",
]



print(f"Number of check-ins: {len(names_list)}")

# Convert all names in names_list to lowercase for consistent comparison
names_list_lower = [name.lower().strip() for name in names_list]



# Load the Excel file and skip the first row to fix headers
file_path = "Attendance Check Example.xlsx"
df = pd.read_excel(file_path, header=1)

# Get the column names
print(df.columns)

# Convert all column names to strings (since pandas might interpret dates as datetime objects)
df.columns = df.columns.astype(str)

# Get the column 'Name' from the Excel file and convert to lowercase
excel_names_lower = df['Name'].str.lower().str.strip().tolist()

# Find names that are in the Excel file but not in names_list
missing_in_names_list = [name for name in excel_names_lower if name not in names_list_lower]

# Find names that are in names_list but not in the Excel file
missing_in_excel_file = [name for name in names_list_lower if name not in excel_names_lower]

print("Names in Excel but not in names_list:", missing_in_names_list)
print("Names in names_list but not in Excel:", missing_in_excel_file)



# Only select Columns 'Name' and date in df that needs to be updated
date = '2024-10-07 00:00:00'
df = df[['Name', date]] 

# empty the column values in the date column
df[date] = ""


# Loop through names_list and check if each name exists in df['Name'] 

for name in names_list:
    # Use loc to select rows where the name matches and update the '2024-09-12 00:00:00' column  使用 loc 和 str.contains 来选择包含 name 的行，并更新 '2024-09-12 00:00:00' 列的值为 'P'
    df.loc[df['Name'].str.contains(name, na=False, case=False), date] = 'P'
    '''
    1) df.loc[...] 是 pandas 的定位器, 用来根据行和列的标签来选择数据 df.loc[row_selection, column_selection]  eg df.loc[0, 'Name']  # 选择第 0 行 'Name' 列的值  df.loc[df['Age'] > 30, 'Name']  # 选择 Age 大于 30 的行的 'Name' 列
    2) df['Name'].str.contains(name, na=False)。 str.contains() 是用于字符串匹配的一个方法。它会检查 df['Name'] 列的每个值是否包含传入的子字符串 name，并返回一个布尔型序列（即每个值是否包含 name，返回 True 或 False）。
        na=False 的作用是将所有的 NaN 值视为 False，从而确保在执行 str.contains() 时，忽略这些空值，并避免因为 NaN 而导致程序出错。因为pandas 的 str.contains() 方法默认会将 NaN 视为 NaN，而不是布尔值。如果数据中存在 NaN，那么 contains() 方法将返回 NaN，这会导致无法进行布尔操作。
        case insensitive 意味着不区分大小写，即不管 name 是大写还是小写，都会被匹配到。    
    3) df.loc[...] = 'P' 是将选中的行的 '2024-09-12 00:00:00' 列的值更新为 'P'
    '''




print(df)

# count the number of 'P' in the '2024-09-12 00:00:00' column
print(f"\n Number of 'P' in {date}: {df[date].str.count('P').sum()}")

# tell the name of the students who are absent
absent_students = df[df[date] != 'P']['Name'].tolist()
print(f"\n Absent students: {absent_students}")


Number of check-ins: 38
Index([              'No.',              'Name',        'Student ID',
       2024-09-12 00:00:00, 2024-09-16 00:00:00, 2024-09-19 00:00:00,
       2024-09-23 00:00:00, 2024-09-26 00:00:00, 2024-09-30 00:00:00,
       2024-10-03 00:00:00, 2024-10-07 00:00:00,       'Unnamed: 11',
             'Unnamed: 12',       'Unnamed: 13',       'Unnamed: 14',
             'Unnamed: 15',       'Unnamed: 16',       'Unnamed: 17'],
      dtype='object')
Names in Excel but not in names_list: ['david', 'eve', 'frank', 'grace']
Names in names_list but not in Excel: ['rudy', 'sara']
       Name 2024-10-07 00:00:00
0     Aaron                   P
1     Alice                   P
2       Amy                   P
3     Bella                   P
4       Bob                   P
5   Charlie                   P
6      Cody                   P
7     David                    
8     Diana                   P
9     Ethan                   P
10      Eve                    
11    Fiona          

# 3. Update the file and keep the rest part intact

In [10]:
from datetime import datetime
from openpyxl import load_workbook

wb = load_workbook(file_path)
ws = wb.active

target_column = None

# Find the target datetime column
for col in ws.iter_cols(1, ws.max_column):
    if str(col[1].value) == date:
        target_column = col[0].column
        break

# Update values if target column is found
if target_column is not None:
    for idx, value in enumerate(df[date], start=3):
        ws.cell(row=idx, column=target_column).value = value

    wb.save(file_path)
    print(f"Data written to '{date}' column and saved to {file_path}")
else:
    print(f"Date column '{date}' not found")


Data written to '2024-10-07 00:00:00' column and saved to Attendance Check Example.xlsx


# 4. Color Absence in Red and accumulate absence times for each person

In [11]:
from openpyxl.styles import PatternFill

# Load Excel workbook
wb = load_workbook(file_path)
ws = wb.active

# Define red fill for absences
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

# Find target column matching 'date'
target_column = None
for col in ws.iter_cols(1, ws.max_column):
    if str(col[1].value) == date:
        target_column = col[0].column
        break

# Update if target column is found
if target_column is not None:
    absence_count = {}
    last_column = ws.max_column + 1
    ws.cell(row=2, column=last_column).value = "Absence Count"

    for idx, value in enumerate(df[date], start=3):
        cell = ws.cell(row=idx, column=target_column)
        student_name = ws.cell(row=idx, column=2).value

        # Mark absent cells in red and update absence count
        if value != 'P':
            cell.fill = red_fill
            absence_count[student_name] = absence_count.get(student_name, 0) + 1
            ws.cell(row=idx, column=last_column).value = absence_count[student_name]
        else:
            ws.cell(row=idx, column=last_column).value = absence_count.get(student_name, 0)

        cell.value = value

    wb.save(file_path)
    print(f"Data written and absences marked in {file_path}")

else:
    print(f"Date column '{date}' not found")


Data written and absences marked in Attendance Check Example.xlsx
